<a href="https://colab.research.google.com/github/blanpa/Tensorflow-Developer-Certificate-Stuff/blob/main/Medical_Cost_Personal_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Content Columns

age: age of primary beneficiary

sex: insurance contractor gender, female, male

bmi: Body mass index, providing an understanding of body, weights that are relatively high or low relative to height,
objective index of body weight (kg / m ^ 2) using the ratio of height to weight, ideally 18.5 to 24.9

children: Number of children covered by health insurance / Number of dependents

smoker: Smoking

region: the beneficiary's residential area in the US, northeast, southeast, southwest, northwest.

charges: Individual medical costs billed by health insurance

In [1]:
!nvidia-smi

Mon May  3 16:36:12 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
import datetime, os

In [3]:
# Import required libaries
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# Read the dataset
insurance = pd.read_csv(filepath_or_buffer = "https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv")

In [5]:
insurance.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [24]:
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [26]:
insurance.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


In [6]:
# One Hot Encoding
insurance_one_hot = pd.get_dummies(data = insurance)
insurance_one_hot

,age,bmi,children,charges,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,1,0,0,1,0,0,0,1
1,18,33.770,1,1725.55230,0,1,1,0,0,0,1,0
2,28,33.000,3,4449.46200,0,1,1,0,0,0,1,0
3,33,22.705,0,21984.47061,0,1,1,0,0,1,0,0
4,32,28.880,0,3866.85520,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,0,1,1,0,0,1,0,0
1334,18,31.920,0,2205.98080,1,0,1,0,1,0,0,0
1335,18,36.850,0,1629.83350,1,0,1,0,0,0,1,0
1336,21,25.800,0,2007.94500,1,0,1,0,0,0,0,1


In [7]:
# Train_test_Split
from sklearn.model_selection import train_test_split

X = insurance_one_hot.drop("charges", axis = 1)
y = insurance_one_hot.charges

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)   

In [11]:
tf.random.set_seed(42)

model1 = tf.keras.Sequential([
    tf.keras.layers.Dense(150),
    tf.keras.layers.Dense(200),
    tf.keras.layers.Dense(150),
    tf.keras.layers.Dense(200),
    tf.keras.layers.Dense(150),
    tf.keras.layers.Dense(100),
    tf.keras.layers.Dense(50),
    tf.keras.layers.Dense(25),
    tf.keras.layers.Dense(5),
    tf.keras.layers.Dense(1)
])

model1.compile(loss=tf.keras.losses.mae,
               optimizer=tf.keras.optimizers.Adam(
                   learning_rate=0.01
               ),
               metrics = [
                    tf.keras.metrics.mean_absolute_error,
                    tf.keras.metrics.mean_squared_error,
                    tf.keras.metrics.hinge])

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

model1.fit(X_train, y_train, epochs = 100, callbacks=[tensorboard_callback])

Epoch 1/100
34/34 [==============================] - 2s 14ms/step - loss: 11128.2366 - mean_absolute_error: 11128.2366 - mean_squared_error: 250135308.3429 - hinge: 0.0000e+00
Epoch 2/100
34/34 [==============================] - 0s 3ms/step - loss: 6562.5935 - mean_absolute_error: 6562.5935 - mean_squared_error: 150899708.3429 - hinge: 523.7532
Epoch 3/100
34/34 [==============================] - 0s 3ms/step - loss: 5938.7454 - mean_absolute_error: 5938.7454 - mean_squared_error: 105950562.5143 - hinge: 14372.8359
Epoch 4/100
34/34 [==============================] - 0s 3ms/step - loss: 5633.1252 - mean_absolute_error: 5633.1252 - mean_squared_error: 71423382.1714 - hinge: 8856.0124
Epoch 5/100
34/34 [==============================] - 0s 3ms/step - loss: 4490.6830 - mean_absolute_error: 4490.6830 - mean_squared_error: 52405692.3429 - hinge: 58765.2868
Epoch 6/100
34/34 [==============================] - 0s 3ms/step - loss: 4147.6570 - mean_absolute_error: 4147.6570 - mean_squared_error:

In [12]:
model1.evaluate(X_test, y_test)

9/9 [==============================] - 0s 3ms/step - loss: 3559.9604 - mean_absolute_error: 3559.9604 - mean_squared_error: 37792416.0000 - hinge: 0.0000e+00


[3559.96044921875, 3559.96044921875, 37792416.0, 0.0]

In [20]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 100), started 0:03:35 ago. (Use '!kill 100' to kill it.)

<IPython.core.display.Javascript object>

In [28]:
from tensorboard import notebook
notebook.list() # View open TensorBoard instances

Known TensorBoard instances:
  - port 6006: logdir logs (started 0:08:06 ago; pid 100)


In [29]:
notebook.display(port=6006, height=1000) 

Selecting TensorBoard with logdir logs (started 0:08:08 ago; port 6006, pid 100).


<IPython.core.display.Javascript object>